In [ ]:
pip install pymysql

  Using cached PyMySQL-1.1.1-py3-none-any.whl.metadata (4.4 kB)
Using cached PyMySQL-1.1.1-py3-none-any.whl (44 kB)
Note: you may need to restart the kernel to use updated packages.


## 1. 작가 이름 JSON에서 찾기
* User 테이블에 기존의 작가를 더미회원으로 삽입하기 위한 우선 작업입니다.


In [ ]:
import os
import zipfile

# 압축 파일들이 들어있는 최상위 폴더
base_dir = "/Users/sopung/Desktop/jupyter_notebook/donghwa"

# 재귀적으로 모든 하위 폴더 탐색
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".zip"):
            zip_path = os.path.join(root, file)
            extract_folder = os.path.join(root, file[:-4])  # .zip 제거한 폴더명

            # 압축 해제
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                print(f"Extracting: {zip_path} → {extract_folder}")
                zip_ref.extractall(extract_folder)


Extracting: /Users/sopung/Desktop/jupyter_notebook/donghwa/Training/VL_05T_신체운동_건강_01S_유아.zip → /Users/sopung/Desktop/jupyter_notebook/donghwa/Training/VL_05T_신체운동_건강_01S_유아
Extracting: /Users/sopung/Desktop/jupyter_notebook/donghwa/Training/TL_01T_의사소통_01S_유아.zip → /Users/sopung/Desktop/jupyter_notebook/donghwa/Training/TL_01T_의사소통_01S_유아
Extracting: /Users/sopung/Desktop/jupyter_notebook/donghwa/Training/TL_04T_예술경험_03S_초등_고학년.zip → /Users/sopung/Desktop/jupyter_notebook/donghwa/Training/TL_04T_예술경험_03S_초등_고학년
Extracting: /Users/sopung/Desktop/jupyter_notebook/donghwa/Training/TL_05T_신체운동_건강_03S_초등_고학년.zip → /Users/sopung/Desktop/jupyter_notebook/donghwa/Training/TL_05T_신체운동_건강_03S_초등_고학년
Extracting: /Users/sopung/Desktop/jupyter_notebook/donghwa/Training/VL_02T_자연탐구_03S_초등_고학년.zip → /Users/sopung/Desktop/jupyter_notebook/donghwa/Training/VL_02T_자연탐구_03S_초등_고학년
Extracting: /Users/sopung/Desktop/jupyter_notebook/donghwa/Training/TL_02T_자연탐구_02S_초등_저학년.zip → /Users/sopung/Desktop/jupyt

In [ ]:
import json

def collect_authors(json_root_dir):
    author_set = set()
    for root, dirs, files in os.walk(json_root_dir):
        for fname in files:
            if fname.endswith(".json"):
                json_path = os.path.join(root, fname)
                try:
                    with open(json_path, "r", encoding="utf-8") as f:
                        data = json.load(f)
                        author = data.get("author", "작자미상").strip()
                        author_set.add(author)
                except Exception as e:
                    print(f"[경고] JSON 파싱 실패: {json_path} | 에러: {e}")
    return sorted(author_set)

# 사용 예
author_dir = '/Users/sopung/Desktop/jupyter_notebook/donghwa/Training'
authors = collect_authors_recursive(author_dir)
print(authors)

['9791157023738', '가 빈', '가만희', '가순열', '가스통 르루', '강두영', '강로사', '강민경', '강민우', '강선남', '강성은', '강성은, 이대형', '강숙인', '강연수', '강용숙', '강은진', '강정규', '강정연', '강정화', '강지혜', '강효미', '강휘생', '겨울', '고래필통', '고선미', '고순덕', '고영이', '고영이 외', '고영이, 하늘땅', '고은경', '고지혜', '고희경', '고희동', '곰세마리', '곽민수', '곽영미', '교연아카데미 편집부', '구들', '권소리', '권윤경', '권정현', '그림 형제', '그림형제', '글깨비', '글뿌리', '기정이', '길선영', '김겨울', '김경란', '김경민', '김경수', '김경희', '김귀경', '김근주', '김남석', '김남중', '김남희', '김동엽', '김두임', '김란주', '김마리아', '김문기', '김미선, 채영숙, 김은정, 박찬', '김미애', '김미연', '김미혜', '김민경', '김민지', '김민진', '김복현', '김복희', '김부식', '김빈애', '김삼진', '김상영', '김서경', '김서연', '김서정', '김서현', '김선영', '김선진', '김성춘', '김세실', '김세영', '김소형', '김수병', '김수정', '김수현', '김수희', '김순한', '김시습', '김연아', '김영', '김영미', '김영숙', '김영숙, 양진희', '김영주', '김영후', '김예림', '김예실', '김옥란', '김완기', '김용란', '김원한', '김유리', '김유리, 아람', '김유리,장선혜', '김윤영', '김윤태', '김은경', '김은미', '김은순', '김은영', '김은정', '김은중', '김은지', '김인숙', '김인숙, 송호정', '김인숙, 아람', '김일광', '김일항', '김자호', '김장성', '김정란', '김정애', '김정하', '김정희', '김주희', '김준영', '김지연', '김지연,김민정', '김지연,김인

In [ ]:
# 총 759개
print(len(authors))

759


## 2. DB연결 확인
* DB에 삽입하기 전 연결을 확인합니다.
* DB의 연결정보는 .env에 있습니다.

In [ ]:
import pymysql
from dotenv import load_dotenv


# .env 파일 로드
load_dotenv()

# DB 접속 정보 (.env에서 불러오기)
DB_CONFIG = {
    'host': os.getenv('DB_HOST'),
    'port': int(os.getenv('DB_PORT', 3306)),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'database': os.getenv('DB_NAME'),
    'charset': 'utf8mb4'
}

def test_db_connection():
    try:
        conn = pymysql.connect(**DB_CONFIG)
        cursor = conn.cursor()
        cursor.execute("SELECT VERSION();")
        version = cursor.fetchone()
        print(f"[✓] MySQL 연결 성공 - 버전: {version[0]}")
        cursor.close()
        conn.close()
    except Exception as e:
        print("[✗] DB 연결 실패:", e)

if __name__ == "__main__":
    test_db_connection()


[✓] MySQL 연결 성공 - 버전: 10.6.22-MariaDB-0ubuntu0.22.04.1


## 3. DB삽입 전 테스트
* 실제 삽입 전 출력으로 테스트 해봅니다.

In [ ]:
# 삽입 전 테스트
def users_preview(cursor, authors):
    author_id_map = {}
    for author in authors:
        # user_id가 PK이므로 수정
        cursor.execute("SELECT user_id FROM User WHERE nickname = %s", (author,))
        result = cursor.fetchone()

        if result:
            print(f"[이미 존재] {author} → User ID {result[0]}")
            author_id_map[author] = result[0]
        else:
            print(f"[삽입 예정] INSERT INTO User (login_id, password_hash, nickname, email, created_at) "
                  f"VALUES (NULL, 'dummy', '{author}', NULL, NOW())")
            fake_id = -(len(author_id_map) + 1)
            author_id_map[author] = fake_id

    return author_id_map


if __name__ == "__main__":
    authors = collect_authors(author_dir)

    conn = pymysql.connect(**DB_CONFIG)
    cursor = conn.cursor()

    print("=== 더미 사용자 삽입 시뮬레이션 (INSERT 없이) ===")
    author_id_map = users_preview(cursor, authors)

    print("\n[가상 Author ID 매핑 결과]")
    for author, fake_id in author_id_map.items():
        print(f"{author} → {fake_id}")

    cursor.close()
    conn.close()


## 4.  작가 목록을 User 테이블에 더미 계정으로 INSERT
* Pelicanic_db.User 테이블에 실제로 데이터를 삽입합니다.


In [ ]:
from datetime import datetime

def insert_dummy_users(cursor, authors):
    author_id_map = {}
    for author in authors:
        cursor.execute("SELECT user_id FROM User WHERE nickname = %s", (author,))
        result = cursor.fetchone()
        if result:
            author_id = result[0]
        else:
            cursor.execute("""
                INSERT INTO User (login_id, password_hash, nickname, email, created_at)
                VALUES (%s, %s, %s, %s, %s)
            """, (None, 'dummy', author, None, datetime.now()))
            author_id = cursor.lastrowid
        author_id_map[author] = author_id
    return author_id_map

if __name__ == "__main__":
    authors = collect_authors(author_dir)

    try:
        conn = pymysql.connect(**DB_CONFIG)
        cursor = conn.cursor()

        author_id_map = insert_dummy_users(cursor, authors)
        conn.commit()

        print("✅ Author → User ID 매핑 결과:")
        for name, uid in author_id_map.items():
            print(f"{name} → {uid}")

    except Exception as e:
        conn.rollback()
        print(f"❌ 에러 발생: {e}")
    finally:
        cursor.close()
        conn.close()


## 5. SUMMARY 생성하기
* Paragraph를 읽어들여 해당 내용을 요약하는 기능을 만들었습니다.
* 추후에 유저에게 보여줄 요약 서비스에 사용되면 좋지않을까 싶네요

In [ ]:
pip install google-generativeai

  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
  Using cached google_api_core-2.24.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_api_python_client-2.170.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached google_auth-2.40.2-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached google_api_core-2.25.0rc1-py3-none-any.whl.metadata (3.0 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl.metadata (9.3 kB)
  Using cached grpcio-1.71.0-cp311-cp311-macosx_10_14_universal2.whl.metadata (3.8 kB)
  Using cached grpcio_status-1.71.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached pyasn1-0.6.1

In [ ]:
import google.generativeai as genai


# 구글 API키 불러오기
try:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    if GOOGLE_API_KEY == "YOUR_GOOGLE_AI_STUDIO_API_KEY":
        print("⚠️ 경고: GOOGLE_API_KEY를 설정해야 합니다. 코드를 수정해주세요.")
    genai.configure(api_key=GOOGLE_API_KEY)
except Exception as e:
    print(f"API 키 설정 중 오류 발생: {e}")
    print("GOOGLE_API_KEY 환경 변수를 설정하거나 코드에 직접 API 키를 입력해주세요.")
    # exit()


gemini_model = genai.GenerativeModel('gemini-1.5-flash-latest')


def generate_summary_llm(paragraphs):
    full_text = "\n".join(p["srcText"] for p in paragraphs)

    # Gemini를 위한 프롬프트
    prompt = f"""당신은 유아동화를 요약하는 작가입니다. 다음은 유아용 동화 전체 본문입니다. 이 내용을 한국어 300자 이내로 줄거리 요약해 주세요.\n\n{full_text[:3500]}""" # 입력 길이 제한은 유지

    try:
        # Gemini API 호출
        response = gemini_model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.7 # 창의성 조절
            )
        )
        return response.text.strip()
    except Exception as e:
        print(f"[요약 실패 - Gemini] {e}")
        return "요약 생성 실패 (Gemini)"


def load_json_and_summarize(json_path):
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        paragraphs = data.get("paragraphInfo", [])
        if not paragraphs:
            raise ValueError("paragraphInfo가 존재하지 않습니다.")

        summary = generate_summary_llm(paragraphs) # 수정된 함수 호출
        print(f"📘 동화 제목: {data.get('title', '제목 없음')}")
        print(f"🖋️ 작가: {data.get('author', '작자 미상')}")
        print(f"📄 Gemini 생성 요약:\n{summary}")

    except json.JSONDecodeError as je:
        print("❌ JSON 파싱 에러:", je)
    except Exception as e:
        print("❌ 처리 중 오류 발생:", e)


if __name__ == "__main__":
    json_file_path = "/Users/sopung/Downloads/Sample/02.라벨링데이터/01T_의사소통/01S_유아/test.json"

    if GOOGLE_API_KEY == "YOUR_GOOGLE_AI_STUDIO_API_KEY":
        print("실행 전에 API 키를 코드에 입력해주세요. (위의 GOOGLE_API_KEY 변수)")
    else:
        if os.path.exists(json_file_path):
            load_json_and_summarize(json_file_path)
        else:
            print(f"❌ 오류: JSON 파일을 찾을 수 없습니다 - {json_file_path}")
            print("실제 JSON 파일 경로로 수정해주세요.")

📘 동화 제목: 몽고메리 버스에서 생긴 일
🖋️ 작가: 한얼교육 편집부
📄 Gemini 생성 요약:
피곤한 몸으로 버스에 오른 흑인 여성 로자 파크스는 백인 남성에게 자리를 양보하라는 버스 운전사의 요구를 거절했습니다.  이는 앨라배마 주의 인종차별적인 버스 좌석 분리법을 정면으로 거부하는 행위였습니다.  파크스 부인은 체포되었지만, 이 사건은 흑인들의 인권 운동에 불을 지폈습니다.  흑인들은 버스 보이콧 운동을 벌이며 백인들의 차별에 맞섰고,  결국 미국 연방 대법원은 앨라배마 주의 인종차별 법을 위헌으로 판결했습니다.  파크스 부인의 용기 있는 행동은 미국 사회의 인종차별 철폐에 중요한 계기가 되었고, 많은 이들에게 희망을 주었습니다.


## 6. TEST JSON 삽입
* 이제 Story, StoryParagraph, ParagraphVersion에 JSON을 읽어들여 동화를 삽입해봅시다.


In [ ]:
from datetime import datetime
from pymysql.converters import escape_string

# 📌 환경 변수(.env) 파일 로딩
# 반드시 루트 디렉토리에 `.env` 파일이 존재해야 하며, Git에 올리지 마세요!
load_dotenv()

# ✅ Gemini API 설정
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")  # 🔒 API 키를 직접 하드코딩하지 마세요. .env에서 불러옵니다.
if not GOOGLE_API_KEY:
    raise ValueError("❌ GOOGLE_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.")

genai.configure(api_key=GOOGLE_API_KEY)
gemini_model = genai.GenerativeModel('gemini-1.5-flash-latest')

# ✅ DB 접속 정보
DB_CONFIG = {
    'host': os.getenv('DB_HOST'),
    'port': int(os.getenv('DB_PORT', 3306)),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'database': os.getenv('DB_NAME'),
    'charset': 'utf8mb4'
}

# ✅ Gemini 요약 함수
def generate_summary_llm(paragraphs):
    full_text = "\n".join(p["srcText"] for p in paragraphs)
    prompt = f"""당신은 유아동화를 요약하는 작가입니다. 다음은 유아용 동화 전체 본문입니다. 이 내용을 한국어 300자 이내로 줄거리 요약해 주세요.\n\n{full_text[:3500]}"""
    try:
        response = gemini_model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(temperature=0.7)
        )
        return response.text.strip()
    except Exception as e:
        print(f"[❌ 요약 실패 - Gemini] {e}")
        return "요약 생성 실패 (Gemini)"

# ✅ 작가 유저 ID 가져오기 또는 생성
def get_or_create_author_user_id(cursor, author_name):
    author_name = author_name.strip()
    cursor.execute("SELECT user_id FROM User WHERE nickname = %s", (author_name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        cursor.execute("""
            INSERT INTO User (login_id, password_hash, nickname, email, created_at)
            VALUES (%s, %s, %s, %s, %s)
        """, (None, 'dummy', author_name, None, now))
        return cursor.lastrowid

# ✅ Story + Paragraph + Version 삽입
def insert_story_and_paragraphs(cursor, json_data):
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    title = escape_string(json_data.get("title", "제목 없음"))
    author_name = json_data.get("author", "작자 미상").strip()
    paragraphs = json_data.get("paragraphInfo", [])
    summary = escape_string(generate_summary_llm(paragraphs))
    author_user_id = get_or_create_author_user_id(cursor, author_name)
    author_name_escaped = escape_string(author_name)

    # 📘 Story 삽입
    cursor.execute(f"""
        INSERT INTO Story (author_user_id, author_name, title, summary, created_at, updated_at, status)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (author_user_id, author_name_escaped, title, summary, now, now, 'published'))
    story_id = cursor.lastrowid
    print(f"[✓] Story '{title}' 삽입 완료 (ID: {story_id})")

    # 📄 Paragraph 및 Version 삽입
    for idx, p in enumerate(paragraphs, 1):
        content = escape_string(p.get("srcText", ""))
        cursor.execute(f"""
            INSERT INTO StoryParagraph (story_id, paragraph_no, content_text, created_at, updated_at)
            VALUES (%s, %s, %s, %s, %s)
        """, (story_id, idx, content, now, now))
        paragraph_id = cursor.lastrowid

        cursor.execute(f"""
            INSERT INTO ParagraphVersion (paragraph_id, version_no, content_text, generated_by, created_at)
            VALUES (%s, %s, %s, %s, %s)
        """, (paragraph_id, 1, content, author_name, now))

    return story_id

# ✅ 메인 실행 함수
def main():
    json_path = "/Users/sopung/Downloads/Sample/02.라벨링데이터/01T_의사소통/01S_유아/test2.json"

    try:
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
    except Exception as e:
        print(f"❌ JSON 로딩 실패: {e}")
        return

    try:
        conn = pymysql.connect(**DB_CONFIG)
        cursor = conn.cursor()
        story_id = insert_story_and_paragraphs(cursor, data)
        conn.commit()
        print(f"[✓] 전체 삽입 완료. Story ID: {story_id}")
    except Exception as e:
        print(f"❌ DB 삽입 실패: {e}")
        conn.rollback()
    finally:
        cursor.close()
        conn.close()

if __name__ == "__main__":
    main()


[✓] Story '사람으로 둔갑 한 쥐' 삽입 완료 (ID: 2)
[✓] 전체 삽입 완료. Story ID: 2


In [ ]:
# DB 값 확인 및 내용삭제용
SELECT * FROM pelicanic_db.User;
SELECT * FROM pelicanic_db.Story;
SELECT * FROM pelicanic_db.StoryParagraph;
SELECT * FROM pelicanic_db.ParagraphVersion;

SET FOREIGN_KEY_CHECKS = 0;

TRUNCATE TABLE pelicanic_db.ParagraphVersion;
TRUNCATE TABLE pelicanic_db.Illustration;
TRUNCATE TABLE pelicanic_db.StoryParagraph;
TRUNCATE TABLE pelicanic_db.Story;

SET FOREIGN_KEY_CHECKS = 1;

## 7. 실제 삽입코드

In [ ]:
# ✅ Gemini API 설정
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")  # 🔒 API 키를 직접 하드코딩하지 마세요. .env에서 불러옵니다.
if not GOOGLE_API_KEY:
    raise ValueError("❌ GOOGLE_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.")

genai.configure(api_key=GOOGLE_API_KEY)
gemini_model = genai.GenerativeModel('gemini-1.5-flash-latest')

# ✅ DB 접속 정보
DB_CONFIG = {
    'host': os.getenv('DB_HOST'),
    'port': int(os.getenv('DB_PORT', 3306)),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'database': os.getenv('DB_NAME'),
    'charset': 'utf8mb4'
}

# ✅ Gemini 요약 함수
def generate_summary_llm(paragraphs):
    full_text = "\n".join(p["srcText"] for p in paragraphs)
    prompt = f"""당신은 유아동화를 요약하는 작가입니다. 다음은 유아용 동화 전체 본문입니다. 이 내용을 한국어 300자 이내로 줄거리 요약해 주세요.\n\n{full_text[:3500]}"""
    try:
        response = gemini_model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(temperature=0.7)
        )
        return response.text.strip()
    except Exception as e:
        print(f"[❌ 요약 실패 - Gemini] {e}")
        return "요약 생성 실패 (Gemini)"

# ✅ 작가 유저 ID 가져오기 또는 생성
def get_or_create_author_user_id(cursor, author_name):
    author_name = author_name.strip()
    cursor.execute("SELECT user_id FROM User WHERE nickname = %s", (author_name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        cursor.execute("""
            INSERT INTO User (login_id, password_hash, nickname, email, created_at)
            VALUES (%s, %s, %s, %s, %s)
        """, (None, 'dummy', author_name, None, now))
        return cursor.lastrowid

# ✅ Story + Paragraph + Version 삽입
def insert_story_and_paragraphs(cursor, json_data):
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    title = escape_string(json_data.get("title", "제목 없음"))
    author_name = json_data.get("author", "작자 미상").strip()
    paragraphs = json_data.get("paragraphInfo", [])
    summary = escape_string(generate_summary_llm(paragraphs))
    author_user_id = get_or_create_author_user_id(cursor, author_name)
    author_name_escaped = escape_string(author_name)

    cursor.execute(f"""
        INSERT INTO Story (author_user_id, author_name, title, summary, created_at, updated_at, status)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (author_user_id, author_name_escaped, title, summary, now, now, 'published'))
    story_id = cursor.lastrowid
    print(f"[✓] Story '{title}' 삽입 완료 (ID: {story_id})")

    for idx, p in enumerate(paragraphs, 1):
        content = escape_string(p.get("srcText", ""))
        cursor.execute(f"""
            INSERT INTO StoryParagraph (story_id, paragraph_no, content_text, created_at, updated_at)
            VALUES (%s, %s, %s, %s, %s)
        """, (story_id, idx, content, now, now))
        paragraph_id = cursor.lastrowid

        cursor.execute(f"""
            INSERT INTO ParagraphVersion (paragraph_id, version_no, content_text, generated_by, created_at)
            VALUES (%s, %s, %s, %s, %s)
        """, (paragraph_id, 1, content, author_name, now))

    return story_id

# ✅ 폴더 내 모든 JSON 처리
def main():
    folder_path = "/Users/sopung/Downloads/Sample/02.라벨링데이터/01T_의사소통/01S_유아"

    try:
        conn = pymysql.connect(**DB_CONFIG)
        cursor = conn.cursor()

        for filename in os.listdir(folder_path):
            if filename.endswith(".json"):
                file_path = os.path.join(folder_path, filename)
                try:
                    with open(file_path, "r", encoding="utf-8") as f:
                        data = json.load(f)
                    story_id = insert_story_and_paragraphs(cursor, data)
                    conn.commit()
                    print(f"[✓] '{filename}' 삽입 성공 → Story ID: {story_id}")
                except Exception as file_error:
                    print(f"[❌] '{filename}' 처리 중 오류: {file_error}")
                    conn.rollback()

    except Exception as db_error:
        print(f"❌ DB 연결 실패: {db_error}")
    finally:
        if 'cursor' in locals(): cursor.close()
        if 'conn' in locals(): conn.close()

if __name__ == "__main__":
    main()
